<a href="https://colab.research.google.com/github/ParagDasAssam/Face_mask_detection/blob/master/object_tracking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

!git clone https://github.com/adipandas/multi-object-tracker.git
    
!pip install numpy matplotlib scipy
!pip install opencv-contrib-python

os.chdir("/home/ubuntu/Desktop/parag/Face_mask_detection/")
!git clone https://github.com/theAIGuysCode/yolov4-deepsort

Cloning into 'or-objectdetection'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 1022 (delta 58), reused 82 (delta 29), pack-reused 896
Receiving objects: 100% (1022/1022), 22.68 MiB | 16.40 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [ ]:
os.chdir("/home/ubuntu/Desktop/parag/Face_mask_detection/multi-object-tracker/")
!pip install -e .

Download the base model

In [ ]:
## Yolo model

%cd ./pretrained_models/yolo_weights
!sudo chmod +x ./get_yolo.sh
!./get_yolo.sh

In [ ]:
# Tensorflow model

os.chdir("/home/ubuntu/Desktop/parag/Face_mask_detection/multi-object-tracker/pretrained_models/tensorflow_weights/")
!sudo chmod +x ./get_ssd_model.sh
!./get_ssd_model.sh

In [ ]:
## Faster_rcnn_inseption_v2 model

os.chdir("/home/ubuntu/Desktop/parag/Face_mask_detection/multi-object-tracker/pretrained_models/")
os.mkdir("faster_rcnn_model")
os.chdir("/home/ubuntu/Desktop/parag/Face_mask_detection/multi-object-tracker/pretrained_models/faster_rcnn_model/")
!wget -c http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz -O - | tar -xz
!wget https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_inception_v2_coco_2018_01_28.pbtxt

## Tracking model

In [1]:
import cv2 as cv
from scipy.spatial import distance
import numpy as np
from collections import OrderedDict

In [2]:
class Tracker:
    def __init__(self, maxLost = 30):           # maxLost: maximum object lost counted when the object is being tracked
        self.nextObjectID = 0                   # ID of next object
        self.objects = OrderedDict()            # stores ID:Locations
        self.lost = OrderedDict()               # stores ID:Lost_count
        
        self.maxLost = maxLost                  # maximum number of frames object was not detected.
        
    def addObject(self, new_object_location):
        self.objects[self.nextObjectID] = new_object_location    # store new object location
        self.lost[self.nextObjectID] = 0                         # initialize frame_counts for when new object is undetected
        
        self.nextObjectID += 1
    
    def removeObject(self, objectID):                          # remove tracker data after object is lost
        del self.objects[objectID]
        del self.lost[objectID]
    
    @staticmethod
    def getLocation(bounding_box):
        xlt, ylt, xrb, yrb = bounding_box
        return (int((xlt + xrb) / 2.0), int((ylt + yrb) / 2.0))
    
    def update(self,  detections):
        
        if len(detections) == 0:   # if no object detected in the frame
            lost_ids = list(self.lost.keys())
            for objectID in lost_ids:
                self.lost[objectID] +=1
                if self.lost[objectID] > self.maxLost: self.removeObject(objectID)
            
            return self.objects
        
        new_object_locations = np.zeros((len(detections), 2), dtype="int")     # current object locations
        print("new_object_locations: ",new_object_locations,"\n")
        
        for (i, detection) in enumerate(detections): new_object_locations[i] = self.getLocation(detection)
            
        if len(self.objects)==0:
            for i in range(0, len(detections)): self.addObject(new_object_locations[i])
        else:
            objectIDs = list(self.objects.keys())
            previous_object_locations = np.array(list(self.objects.values()))
            
            D = distance.cdist(previous_object_locations, new_object_locations) # pairwise distance between previous and current
            
            row_idx = D.min(axis=1).argsort()   # (minimum distance of previous from current).sort_as_per_index
            
            cols_idx = D.argmin(axis=1)[row_idx]   # index of minimum distance of previous from current
            
            assignedRows, assignedCols = set(), set()
            
            for (row, col) in zip(row_idx, cols_idx):
                
                if row in assignedRows or col in assignedCols:
                    continue
                
                objectID = objectIDs[row]
                self.objects[objectID] = new_object_locations[col]
                self.lost[objectID] = 0
                
                assignedRows.add(row)
                assignedCols.add(col)
                
            unassignedRows = set(range(0, D.shape[0])).difference(assignedRows)
            unassignedCols = set(range(0, D.shape[1])).difference(assignedCols)
            
            
            if D.shape[0]>=D.shape[1]:
                for row in unassignedRows:
                    objectID = objectIDs[row]
                    self.lost[objectID] += 1
                    
                    if self.lost[objectID] > self.maxLost:
                        self.removeObject(objectID)
                        
            else:
                for col in unassignedCols:
                    self.addObject(new_object_locations[col])
            
        return self.objects

In [13]:
model_info = dict(    
  config_path="/home/ubuntu/Desktop/parag/Face_mask_detection/multi-object-tracker/pretrained_models/faster_rcnn_model/faster_rcnn_inception_v2_coco_2018_01_28.pbtxt",
  model_weights_path="/home/ubuntu/Desktop/parag/Face_mask_detection/multi-object-tracker/pretrained_models/faster_rcnn_model/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb",
  object_names={0: 'background',
                1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
                7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
                13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
                18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
                24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
                32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
                37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
                41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
                46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
                51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
                56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
                61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
                67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
                75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
                80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
                86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush'},
  confidence_threshold=0.7,
  threshold=0.4
)

net = cv.dnn.readNetFromTensorflow(model_info["model_weights_path"], model_info["config_path"])

In [4]:
np.random.seed(12345)

bbox_colors = {key: np.random.randint(0, 255, size=(3,)).tolist() for key in model_info['object_names'].keys()}

In [5]:
# video_src="/home/ubuntu/Desktop/parag/Face_mask_detection/yolov4-deepsort/data/video/test.mp4"#0
video_scr = "/home/ubuntu/Desktop/MTW-CCTVVideos/gun-manpoint.mp4"
cap = cv.VideoCapture(video_scr)
print(cap)

<VideoCapture 0x7f057d373170>


## Model code

In [23]:
import numpy as np
import tensorflow as tf
import cv2
import time

(H, W) = (300, 300)  # input image height and width for the network
writer = None
maxLost = 5   # maximum number of object losts counted when the object is being tracked
tracker = Tracker(maxLost = maxLost)

class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()
        
PATH_TO_CKPT =  '/home/ubuntu/Desktop/Vivek/Social-Distancing&Mask-Detection/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = '/home/ubuntu/Desktop/demo.pbtxt'


if __name__ == "__main__":
    model_path = PATH_TO_CKPT
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.7
    cap = cv2.VideoCapture('/home/ubuntu/Desktop/MTW-CCTVVideos/gun-manpoint.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/home/ubuntu/Desktop/Swasti/new_p3_output.avi',fourcc, 20.0, (1280,720))

    while True:
        r, img = cap.read()  
        
        img = cv2.resize(img, (700, 500))

        boxes, scores, classes, num = odapi.processFrame(img)
        

        

#         indices = np.argwhere(classes == 1)
#         boxes = np.squeeze(boxes[indices], axis=1) # to prevent errors made by nd.array of size 1 nd.array
#         scores = np.squeeze(scores[indices], axis=1)
#         classes = np.squeeze(classes[indices], axis=1)
#         num = len(scores)


        # Visualization of the results of a detection.
        
#         print("boxes: ",boxes)
#         print("scores",scores)
#         print("classes",classes)
#         print("num",num,"\n")
        
        scores = [x for x in scores if x > threshold]
        boxes = [boxes[x] for x in range(len(scores))]
        classes = [classes[x] for x in range(len(scores))]
        num = len(scores)
        print("boxes: ",boxes)
        print("scores",scores)
        print("classes",classes)
        print("num",num,"\n")

        for i in range(len(boxes)):
            # Class 1 represents human
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)

        objects = tracker.update(boxes)           # update tracker based on the newly detected objects
        print("Object: ",objects,"\n")
        for (objectID, centroid) in objects.items():
            text = "ID {}".format(objectID)
            cv.putText(img, text, (centroid[0] - 10, centroid[1] - 10), cv.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)
            cv.circle(img, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

        cv.imshow("image",img)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

        if writer is None:
            fourcc = cv.VideoWriter_fourcc(*"MJPG")
            
            writer = cv.VideoWriter("output.avi", fourcc, 30, (W, H), True)
        writer.write(img)

writer.release()
cap.release()
# cv.destroyWindow("image")

   #video.release()
cv2.destroyAllWindows()

Elapsed Time: 3.728364944458008
boxes:  [(157, 462, 395, 555), (146, 385, 386, 471), (16, 191, 121, 219), (73, 445, 230, 506), (54, 380, 176, 440), (11, 113, 81, 139)]
scores [0.9992790818214417, 0.997969925403595, 0.9744880795478821, 0.9678480625152588, 0.8872302770614624, 0.8866495490074158]
classes [1, 1, 1, 1, 1, 1]
num 6 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(0, array([276, 508])), (1, array([266, 428])), (2, array([ 68, 205])), (3, array([151, 475])), (4, array([115, 410])), (5, array([ 46, 126]))]) 

Elapsed Time: 0.14257502555847168
boxes:  [(154, 464, 383, 561), (141, 389, 384, 477), (16, 191, 121, 219), (11, 118, 81, 145), (75, 447, 225, 506), (62, 381, 189, 444), (31, 349, 104, 385), (18, 332, 99, 360)]
scores [0.9991982579231262, 0.9946146607398987, 0.9754396080970764, 0.9560431838035583, 0.9386682510375977, 0.8544184565544128, 0.7503560781478882, 0.7301339507102966]
classes [1, 1, 1, 1, 1, 1, 1, 1]
num 8 

new_object_lo

Elapsed Time: 0.1399374008178711
boxes:  [(122, 483, 341, 559), (110, 402, 344, 490), (17, 191, 121, 218), (14, 145, 84, 175), (29, 349, 103, 382), (53, 399, 161, 457), (32, 386, 152, 425), (62, 455, 181, 503)]
scores [0.9992954730987549, 0.997209370136261, 0.9765466451644897, 0.9616557955741882, 0.8664777874946594, 0.8586050271987915, 0.8516265153884888, 0.7845545411109924]
classes [1, 1, 1, 1, 1, 1, 1, 1]
num 8 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(0, array([231, 521])), (1, array([227, 446])), (2, array([ 69, 204])), (3, array([121, 479])), (4, array([107, 428])), (5, array([ 49, 160])), (6, array([ 66, 365])), (8, array([ 92, 405]))]) 

Elapsed Time: 0.14116621017456055
boxes:  [(118, 480, 336, 560), (107, 405, 346, 491), (15, 190, 120, 220), (16, 147, 86, 176), (28, 350, 105, 385), (30, 387, 155, 425), (62, 455, 177, 503), (53, 401, 165, 457)]
scores [0.999116837978363, 0.996737539768219, 0.9719437956809998, 0.95

Elapsed Time: 0.1564183235168457
boxes:  [(99, 484, 298, 560), (81, 420, 297, 496), (17, 186, 121, 225), (13, 333, 93, 362), (54, 392, 175, 451)]
scores [0.9987967014312744, 0.9954349398612976, 0.9799222946166992, 0.9382607936859131, 0.9012709259986877]
classes [1, 1, 1, 1, 1]
num 5 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(0, array([198, 522])), (1, array([189, 458])), (2, array([ 69, 205])), (4, array([114, 421])), (5, array([ 57, 184])), (6, array([ 53, 347])), (8, array([ 93, 398]))]) 

Elapsed Time: 0.14164352416992188
boxes:  [(96, 485, 290, 560), (80, 422, 286, 497), (17, 187, 120, 226), (53, 388, 173, 451), (466, 431, 500, 480)]
scores [0.99909508228302, 0.9937447905540466, 0.9918033480644226, 0.9097334146499634, 0.8545384407043457]
classes [1, 1, 1, 1, 1]
num 5 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(0, array([193, 522])), (1, array([183, 459])), (2, array([ 68, 206])), (4, array([113, 41

Elapsed Time: 0.14283037185668945
boxes:  [(350, 371, 498, 501), (83, 473, 261, 539), (71, 423, 262, 484), (18, 216, 113, 259), (16, 194, 119, 223), (48, 391, 169, 440), (13, 341, 87, 377), (23, 366, 119, 410)]
scores [0.99772709608078, 0.9973725080490112, 0.9897823333740234, 0.9741885662078857, 0.9458461403846741, 0.8698082566261292, 0.8474811911582947, 0.7708334922790527]
classes [1, 1, 1, 1, 1, 1, 1, 1]
num 8 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(0, array([172, 506])), (1, array([166, 453])), (2, array([ 67, 208])), (4, array([108, 415])), (6, array([ 50, 359])), (9, array([424, 436])), (10, array([ 71, 388])), (11, array([ 65, 237]))]) 

Elapsed Time: 0.14450407028198242
boxes:  [(342, 370, 498, 505), (82, 472, 263, 537), (72, 422, 259, 483), (19, 216, 112, 260), (16, 194, 119, 223), (41, 390, 170, 436), (11, 342, 85, 378), (22, 366, 122, 407)]
scores [0.9976630210876465, 0.9971165657043457, 0.9887229204177856, 0.

Elapsed Time: 0.14391350746154785
boxes:  [(263, 366, 494, 470), (71, 472, 243, 532), (20, 246, 121, 297), (60, 412, 239, 475), (18, 191, 121, 224), (33, 384, 167, 436), (23, 351, 105, 387)]
scores [0.9993656277656555, 0.9975569248199463, 0.9951562881469727, 0.9912189841270447, 0.9752006530761719, 0.9597062468528748, 0.9177197217941284]
classes [1, 1, 1, 1, 1, 1, 1]
num 7 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(0, array([157, 502])), (1, array([149, 443])), (2, array([ 69, 207])), (4, array([100, 410])), (9, array([378, 418])), (10, array([ 64, 369])), (11, array([ 70, 271]))]) 

Elapsed Time: 0.140869140625
boxes:  [(255, 366, 495, 470), (70, 472, 243, 532), (21, 248, 122, 297), (61, 408, 238, 475), (19, 349, 107, 385), (103, 549, 257, 599), (40, 386, 165, 437), (16, 191, 120, 225)]
scores [0.9994631409645081, 0.9968196153640747, 0.9960055947303772, 0.9930536150932312, 0.9365808367729187, 0.841870903968811, 0.802793562412262,

...

Elapsed Time: 0.1461772918701172
boxes:  [(121, 230, 303, 300), (257, 365, 492, 478), (292, 455, 493, 554), (116, 515, 319, 593), (16, 107, 119, 149), (107, 469, 300, 540), (41, 389, 169, 430), (51, 425, 189, 481)]
scores [0.9995291233062744, 0.9981335997581482, 0.9963730573654175, 0.9957661628723145, 0.9850178360939026, 0.9574288725852966, 0.9051885604858398, 0.8246456384658813]
classes [1, 1, 1, 1, 1, 1, 1, 1]
num 8 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(2, array([ 67, 128])), (41, array([120, 453])), (47, array([ 56, 372])), (50, array([217, 554])), (52, array([212, 265])), (53, array([203, 504])), (54, array([374, 421])), (55, array([105, 409])), (57, array([392, 504]))]) 

Elapsed Time: 0.14026904106140137
boxes:  [(121, 230, 306, 304), (253, 366, 492, 480), (118, 519, 313, 598), (288, 453, 495, 556), (16, 107, 119, 148), (105, 461, 298, 541), (37, 390, 164, 432), (51, 425, 183, 482)]
scores [0.9992877840995789, 0

Elapsed Time: 0.1409003734588623
boxes:  [(146, 545, 352, 625), (216, 371, 475, 474), (243, 450, 492, 542), (18, 104, 116, 144), (113, 275, 284, 323), (38, 389, 158, 433), (106, 465, 268, 521), (128, 491, 293, 561), (56, 441, 175, 486), (117, 477, 281, 545)]
scores [0.997747004032135, 0.9975405931472778, 0.9965561628341675, 0.9920011758804321, 0.988690197467804, 0.9419898390769958, 0.9188541173934937, 0.9069029092788696, 0.8099731802940369, 0.7053366303443909]
classes [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num 10 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(2, array([ 67, 124])), (41, array([115, 463])), (50, array([249, 585])), (52, array([198, 299])), (53, array([210, 526])), (54, array([345, 422])), (55, array([ 98, 411])), (57, array([367, 496])), (60, array([187, 493]))]) 

Elapsed Time: 0.1422889232635498
boxes:  [(148, 549, 350, 628), (215, 371, 479, 473), (242, 448, 491, 542), (19, 105, 117, 144), (122, 275, 28

Elapsed Time: 0.14075422286987305
boxes:  [(184, 368, 420, 461), (178, 563, 392, 648), (429, 551, 495, 631), (213, 444, 466, 543), (149, 517, 361, 591), (429, 378, 498, 461), (29, 383, 158, 426), (19, 110, 115, 145), (96, 304, 260, 369), (122, 467, 252, 529), (37, 417, 164, 463), (55, 457, 165, 498), (314, 356, 389, 387)]
scores [0.9990241527557373, 0.9961077570915222, 0.990432858467102, 0.9889304041862488, 0.9702724814414978, 0.9668171405792236, 0.9637313485145569, 0.9455434679985046, 0.9448426961898804, 0.9334575533866882, 0.9334309697151184, 0.9167197346687317, 0.7260004878044128]
classes [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 31]
num 13 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(2, array([ 67, 127])), (41, array([110, 477])), (50, array([285, 605])), (52, array([178, 336])), (53, array([255, 554])), (54, array([302, 414])), (55, array([ 93, 404])), (57, array([339, 493])), (60, array([1

Elapsed Time: 0.1410236358642578
boxes:  [(190, 441, 418, 528), (174, 523, 391, 610), (14, 114, 116, 154), (205, 572, 418, 660), (165, 375, 400, 457), (89, 331, 242, 391), (388, 546, 498, 647), (375, 371, 498, 451), (424, 475, 497, 561), (49, 450, 163, 490), (37, 410, 157, 455), (123, 469, 233, 529), (30, 382, 150, 421)]
scores [0.9963900446891785, 0.9942532181739807, 0.9932705760002136, 0.9930742979049683, 0.9912972450256348, 0.9756749272346497, 0.9728075265884399, 0.9657101035118103, 0.9160557389259338, 0.9087678790092468, 0.8987081050872803, 0.8965062499046326, 0.8439263701438904]
classes [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num 13 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(2, array([ 65, 134])), (41, array([106, 470])), (50, array([311, 616])), (52, array([165, 361])), (53, array([282, 566])), (54, array([282, 416])), (55, array([ 90, 401])), (57, array([304, 484])), (60, array([17

Elapsed Time: 0.14456820487976074
boxes:  [(227, 601, 441, 690), (170, 442, 403, 519), (146, 374, 369, 457), (14, 114, 115, 150), (205, 525, 396, 625), (320, 342, 497, 435), (129, 482, 294, 543), (66, 349, 236, 406), (41, 443, 164, 483), (33, 402, 147, 451), (357, 539, 497, 630), (388, 460, 497, 555)]
scores [0.9953877329826355, 0.9947698712348938, 0.9941364526748657, 0.9885392785072327, 0.9719973206520081, 0.9590784907341003, 0.939293622970581, 0.8744370937347412, 0.868533730506897, 0.811888575553894, 0.7957488298416138, 0.7950548529624939]
classes [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num 12 

new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Object:  OrderedDict([(2, array([ 64, 132])), (41, array([102, 463])), (50, array([334, 645])), (52, array([151, 377])), (53, array([300, 575])), (54, array([257, 415])), (55, array([ 87, 400])), (57, array([286, 480])), (60, array([211, 512])), (61, array([ 90, 426])), (63, array([427, 

In [22]:
#https://gist.github.com/madhawav/1546a4b99c8313f06c0b2d7d7b4a09e2

# Code adapted from Tensorflow Object Detection Framework
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
# Tensorflow Object Detection Detector

import numpy as np
import tensorflow as tf
import cv2
import time


class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()
        
PATH_TO_CKPT =  '/home/ubuntu/Desktop/Vivek/Social-Distancing&Mask-Detection/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = '/home/ubuntu/Desktop/demo.pbtxt'


if __name__ == "__main__":
    model_path = PATH_TO_CKPT
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.7
    cap = cv2.VideoCapture('/home/ubuntu/Desktop/MTW-CCTVVideos/gun-manpoint.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/home/ubuntu/Desktop/Swasti/new_p3_output_2.avi',fourcc, 20.0, (1280,720))

    while True:
        r, img = cap.read()
        img = cv2.resize(img, (1280, 720))
        detections = []

        boxes, scores, classes, num = odapi.processFrame(img)

        # Visualization of the results of a detection.

        for i in range(len(boxes)):
            # Class 1 represents human
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
                detections.append((x, y, x+w, y+h))
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)

        print("Detections", detections)

        objects = tracker.update(detections)           # update tracker based on the newly detected objects
    
        for (objectID, centroid) in objects.items():
            text = "ID {}".format(objectID)
            cv.putText(image, text, (centroid[0] - 10, centroid[1] - 10), cv.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 255, 0), 2)
            cv.circle(image, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

        cv2.imshow("/home/ubuntu/Desktop/Swasti/preview.avi", img)
        out.write(img)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break

out.release()
cap.release()
   #video.release()
cv2.destroyAllWindows()


Elapsed Time: 3.575312852859497
Detections [(198, 702, 763, 1563), (221, 840, 788, 1854), (19, 343, 186, 743), (92, 814, 423, 1737), (6, 197, 112, 443), (17, 608, 152, 1261), (74, 687, 334, 1500)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16227936744689941
Detections [(218, 845, 772, 1865), (197, 705, 746, 1570), (19, 344, 186, 743), (7, 208, 113, 464), (90, 816, 423, 1743), (72, 692, 330, 1509)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.15923285484313965
Detections [(205, 846, 759, 1872), (184, 705, 738, 1573), (19, 344, 186, 743), (5, 213, 119, 483), (88, 820, 419, 1748), (81, 694, 328, 1513), (18, 608, 156, 1267)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16181445121765137
Detections [(202, 853, 746, 1882), (188, 710, 726, 1578), (19, 343, 186, 742), (7, 216, 118, 500), (89, 825, 410, 1750), (79, 699, 323, 1514)]
new_object_locations:  [[0 0]
 [

Elapsed Time: 0.16472101211547852
Detections [(128, 884, 544, 1898), (110, 760, 511, 1666), (19, 340, 186, 759), (626, 760, 1341, 1614), (17, 607, 142, 1269), (63, 704, 304, 1530)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1580212116241455
Detections [(118, 876, 531, 1885), (109, 763, 510, 1668), (594, 698, 1308, 1591), (17, 345, 180, 777), (10, 605, 137, 1269), (58, 720, 312, 1536), (21, 659, 254, 1407)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.17332768440246582
Detections [(118, 870, 532, 1876), (109, 764, 511, 1666), (578, 691, 1289, 1581), (16, 347, 181, 774), (55, 716, 294, 1533), (13, 608, 138, 1274)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1667189598083496
Detections [(129, 869, 539, 1870), (107, 763, 507, 1662), (564, 676, 1277, 1567), (22, 347, 187, 751), (11, 608, 134, 1276), (17, 360, 177, 798), (53, 723, 296, 1536)]
new_object_locations:  [

Elapsed Time: 0.1703791618347168
Detections [(322, 676, 1003, 1562), (92, 842, 429, 1791), (75, 732, 405, 1574), (17, 347, 182, 744), (131, 986, 504, 2046), (26, 481, 201, 1029)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16990375518798828
Detections [(319, 686, 992, 1574), (87, 836, 420, 1785), (133, 978, 504, 2036), (26, 486, 201, 1033), (73, 734, 399, 1577), (17, 348, 182, 745)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1638336181640625
Detections [(305, 691, 970, 1578), (87, 836, 420, 1784), (133, 965, 502, 2021), (27, 494, 200, 1050), (71, 736, 391, 1578), (17, 347, 183, 744)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1669003963470459
Detections [(292, 701, 937, 1586), (84, 832, 412, 1778), (131, 943, 501, 1995), (17, 347, 182, 744), (25, 494, 200, 1054), (73, 738, 386, 1580), (41, 617, 231, 1292)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [

Elapsed Time: 0.16057062149047852
Detections [(165, 744, 611, 1629), (35, 485, 211, 1040), (22, 347, 187, 747), (18, 607, 154, 1269), (22, 690, 220, 1460), (93, 835, 325, 1747), (65, 735, 300, 1561)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1639235019683838
Detections [(156, 744, 597, 1627), (33, 486, 209, 1034), (21, 347, 187, 746), (20, 606, 153, 1270), (22, 693, 221, 1458), (92, 835, 318, 1744)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16120576858520508
Detections [(151, 747, 586, 1629), (34, 483, 208, 1026), (22, 347, 187, 746), (21, 606, 155, 1275), (64, 737, 291, 1568), (22, 697, 211, 1463), (89, 833, 342, 1743)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16199994087219238
Detections [(143, 748, 571, 1634), (30, 477, 203, 1017), (22, 347, 187, 746), (18, 607, 151, 1275), (83, 828, 349, 1737), (60, 742, 297, 1571), (23, 703, 198, 1472)]
new_o

Elapsed Time: 0.15720534324645996
Detections [(99, 769, 435, 1656), (31, 434, 204, 925), (21, 345, 185, 743), (9, 603, 133, 1259), (48, 702, 277, 1473)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.15792202949523926
Detections [(95, 768, 429, 1658), (32, 435, 205, 925), (21, 345, 185, 743), (54, 699, 284, 1464), (11, 604, 132, 1258)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1616044044494629
Detections [(96, 772, 426, 1662), (31, 436, 204, 927), (21, 344, 185, 741), (59, 698, 290, 1464), (9, 604, 131, 1256), (38, 759, 243, 1587)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1625375747680664
Detections [(94, 772, 423, 1659), (31, 436, 202, 926), (21, 345, 185, 741), (62, 699, 294, 1466), (11, 603, 133, 1255), (34, 760, 245, 1584), (15, 637, 159, 1319)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.15889787673950195
Detections [(94, 770, 4

...

Elapsed Time: 0.16943573951721191
Detections [(310, 616, 960, 1406), (26, 153, 188, 407), (301, 848, 917, 1865), (457, 641, 1161, 1452), (198, 893, 566, 1909), (66, 733, 311, 1568), (89, 820, 393, 1767)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1587817668914795
Detections [(25, 154, 187, 408), (406, 633, 1110, 1433), (317, 623, 956, 1410), (291, 848, 898, 1873), (83, 818, 396, 1762), (68, 734, 310, 1566), (196, 889, 562, 1904)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16532683372497559
Detections [(435, 635, 1142, 1441), (26, 155, 188, 409), (85, 821, 395, 1760), (290, 845, 897, 1869), (72, 735, 314, 1566), (196, 890, 543, 1899), (340, 627, 995, 1422), (177, 737, 659, 1605)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1550135612487793
Detections [(26, 156, 189, 408), (436, 643, 1147, 1452), (292, 848, 886, 1869), (320, 629, 950, 1423)

Elapsed Time: 0.1688380241394043
Detections [(29, 154, 189, 409), (409, 890, 1124, 1972), (368, 732, 1083, 1679), (198, 857, 644, 1840), (67, 719, 306, 1525), (220, 686, 781, 1524), (344, 702, 976, 1541), (384, 1034, 979, 2163)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16256093978881836
Detections [(407, 890, 1122, 1972), (29, 153, 189, 409), (197, 854, 637, 1838), (64, 717, 306, 1522), (329, 704, 969, 1552), (328, 729, 1037, 1682), (233, 691, 793, 1527), (372, 1027, 980, 2158)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.17033958435058594
Detections [(29, 152, 189, 407), (410, 888, 1122, 1969), (62, 716, 311, 1523), (317, 705, 966, 1545), (189, 850, 621, 1830), (291, 734, 984, 1692), (222, 695, 714, 1531)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16460037231445312
Detections [(404, 886, 1119, 1967), (28, 152, 190, 405), (59, 7

Elapsed Time: 0.16926884651184082
Detections [(475, 629, 1190, 1440), (286, 1034, 886, 2187), (21, 159, 180, 414), (273, 769, 934, 1708), (154, 904, 521, 1930), (516, 846, 1235, 1889), (119, 852, 402, 1808), (309, 924, 892, 2000), (67, 711, 295, 1498), (205, 741, 650, 1595)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16988539695739746
Detections [(282, 1036, 872, 2190), (21, 159, 181, 415), (464, 625, 1176, 1437), (515, 845, 1231, 1889), (500, 1090, 1210, 2333), (279, 769, 937, 1702), (62, 706, 289, 1494), (118, 852, 403, 1812), (153, 900, 536, 1925), (335, 925, 919, 2008), (193, 738, 626, 1599)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16497063636779785
Detections [(457, 623, 1167, 1433), (20, 159, 180, 415), (476, 1070, 1177, 2316), (279, 1037, 856, 2190), (274, 777, 935, 1716), (506, 839, 1226, 1884), (157, 905, 533, 1930), (61, 706, 284,

Elapsed Time: 0.16330671310424805
Detections [(190, 784, 743, 1718), (25, 158, 186, 414), (357, 641, 963, 1447), (489, 1047, 1198, 2280), (233, 944, 745, 2036), (353, 1039, 952, 2245), (392, 853, 985, 1889), (55, 695, 263, 1455), (104, 957, 393, 1985), (99, 843, 300, 1769), (48, 773, 211, 1632), (131, 890, 441, 1867), (119, 758, 480, 1612)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1574099063873291
Detections [(23, 158, 184, 415), (189, 776, 733, 1709), (469, 1046, 1179, 2274), (230, 945, 741, 2040), (365, 1039, 1012, 2255), (386, 861, 971, 1899), (344, 643, 936, 1456), (53, 694, 259, 1455), (105, 954, 393, 1980), (125, 890, 475, 1868)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16185283660888672
Detections [(471, 1042, 1181, 2266), (22, 157, 183, 414), (180, 780, 717, 1712), (380, 862, 950, 1899), (341, 1044, 927, 2253), (229, 

Elapsed Time: 0.1636979579925537
Detections [(22, 156, 182, 406), (138, 778, 483, 1689), (448, 721, 1158, 1655), (169, 912, 516, 1960), (102, 723, 427, 1544)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16165590286254883
Detections [(23, 156, 185, 405), (136, 781, 473, 1690), (438, 733, 1136, 1701), (171, 909, 515, 1958)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.15747404098510742
Detections [(25, 156, 186, 406), (135, 777, 463, 1686), (91, 722, 409, 1544), (425, 731, 1135, 1651), (174, 915, 506, 1961), (308, 969, 912, 2067)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.1556253433227539
Detections [(25, 155, 185, 407), (132, 773, 455, 1678), (240, 1060, 675, 2187), (295, 1010, 866, 2106), (73, 718, 378, 1541), (168, 913, 487, 1962)]
new_object_locations:  [[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]] 

Elapsed Time: 0.16231703758239746
Detections [(24, 156, 186, 409), (161, 914, 481, 1956),